# POS tagger model

## Load data from JSON

In [8]:
import numpy as np
import json
import os

In [9]:
files = [os.path.join('json/gc/webtext/json',x) for x in os.listdir('json/gc/webtext/json')]
corpus = []

for x in files:
    with open(x,'r',encoding='utf-16') as f:
#         print('opening {0}'.format(x))
        doc = json.load(f)
        for i in doc.keys():
#             print("key {0} of document {1}".format(i,x))
            try:
#                 print(doc[i][0][0])
                if (len(doc[i][0][0])<200  and len(i) >0): #150 190 gave same max len
                       corpus.append(doc[i][0][0])  #append each sentence to corpus
            except:
# #                 print(doc[i])
                continue

In [10]:


files = [os.path.join('json/gc/books',x) for x in os.listdir('json/gc/books')]
for x in files:
    with open(x,'r',) as f:
#         print('opening {0}'.format(x))
        doc = json.load(f)
#         for i in doc.keys():
        doc = doc ['ps']
        for i in doc:
            if len(i) <200 and len(i) >0:
                corpus.append(i)
#                 print(i)
            
%reset_selective -f doc
#     break

In [11]:
print(len(corpus))

103566


In [12]:
sentences = []
sentence_tags = []

for sentence in corpus:
    x=[]
    y=[]
    for word in sentence:
        x.append(word[1])
        y.append(word[0])
    if len(x) > 0:
        sentences.append(x)
        sentence_tags.append(y)
# del corpus

In [13]:
print("Tagged_sentences",len(sentence_tags))

Tagged_sentences 103477


In [14]:
print(np.array(sentences[5]).shape)
print(np.array(sentence_tags[5]).shape)

(12,)
(12,)


## Replace with stems

In [ ]:
from nepsent.Stemmer import Stemmer 
stemmer = Stemmer()


In [ ]:
len(sentences)

In [ ]:
exception_tags = ['NP','YF','YM','YQ','YB','FF','FS']
for i,tarsentence in enumerate(sentences):
    for j,word in enumerate(tarsentence):
        if type(word) !=type('str'):
#             print(word)
            sentences[i][j] = str(word)
#             del sentence_tags[i][j] 
            continue
        if sentence_tags[i][j] not  in exception_tags:
            sentences[i][j] = stemmer.stem_word(word)
#             print(sentences[i][j],sentence_tags[i][j],stemmer.stem_word(word))           
#           print(sentences[i][j])  

## Convert Labels to numbers

In [15]:
labels = set()
for sent in sentence_tags:
    for w in sent:
        labels.add(w)

In [16]:
len(labels)

109

In [17]:
for i,sentence in enumerate(sentences):
    for j,word in enumerate(sentence):
        if type(word) !=type('str'):
            sentences[i][j] = str(word)
#             print(word)

In [18]:
import math
# from keras.utils import np_utils

tag2index = {t:i+1 for i,t in enumerate(list(labels))}
tag2index['-PAD-']=0

for i,x in enumerate(sentence_tags):
    sentence_tags[i] = [tag2index[t] for t in x]
 

## Convert word 2 vec and encode labels

In [19]:
import gensim.models.word2vec as w2v
# import multiprocessing


In [ ]:
nepali2vector = w2v.Word2Vec(sentences,size=150,window=10,min_count=3,workers=multiprocessing.cpu_count())

In [ ]:
nepali2vector.train(sentences,total_examples=len(sentences),epochs=10)
# nepali2vector.t

In [20]:
# nepali2vector.save('nepw2vmodel')
nepw2v=w2v.Word2Vec.load('w2vmodel')

In [ ]:
len(nepw2v.wv.vocab)

In [ ]:
nepw2v.wv.most_similar(positive =['केटा','बहिनी'],negative=['केटी'])

In [21]:
sentences_X = []
sentence_tags_Y = [] 
for i,sentence in enumerate(sentences):
    if len(sentence) > 0:
        sent = []
        sent_tag=[]
        for j,word in enumerate(sentence):
            try:
                sent.append(nepw2v.wv.get_vector(word))
                sent_tag.append(sentence_tags[i][j])
            except:
                continue

        sentences_X.append(np.array(sent))
        sentence_tags_Y.append(np.array(sent_tag))
sentences_X = np.array(sentences_X)
sentence_tags_Y = np.array(sentence_tags_Y)
# train_sentences,test_sentences,train_sentence_tags,test_sentence_tags  = train_test_split(sentences,sentence_tags,test_size=0.2)

In [ ]:
se

In [ ]:
tags = []
for sentence in sentence_tags:
    for tag in sentence:
        tags.append(tag)

In [ ]:
from collections import Counter
tags_counts = Counter(tags)
tags_counts['NN']/len

In [ ]:
tags_counts

In [ ]:
%matplotlib qt
import pandas as pd
df = pd.DataFrame.from_dict(tags_counts,orient='index')
df.plot(kind='bar')


In [ ]:
from matplotlib import pyplot as plt

plt.hist(tags)
plt.show()

## Train Test split

In [22]:
from sklearn.model_selection import train_test_split
# skf = StratifiedKFold(n_splits=3,shuffle=True)
train_sentences,test_sentences,train_sentence_tags,test_sentence_tags = train_test_split(sentences_X,sentence_tags_Y,test_size=0.2)

In [23]:
MAX_LENGTH = len(max(sentences_X,key=len))

In [24]:
MAX_LENGTH

148

## padding seq

In [ ]:
# for i,sentence in enumerate(train_sentences):
# #     print(len(sentence))
#     if len(sentence) < MAX_LENGTH:
#         for x in range(MAX_LENGTH-len(sentence)):
#             train_sentences[i]=np.append(train_sentences[i],np.zeros((300,).T),axis=0)
#             train_sentence_tags[i]=np.append(train_sentence_tags[i],np.zeros((1,)),axis=0)
#             test_sentences[i]=np.append(test_sentences[i],np.zeros((300,).T),axis=0)
#             test_sentence_tags[i]=np.append(test_sentence_tags[i],np.zeros((1,)),axis=0)
            
# #         break
# #     print(len(sentence))
# #     break

## Using generator

In [25]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
#             print(item)
            item = item.astype(int)
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [26]:
import math
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

train_sentences,test_sentences,train_sentence_tags,test_sentence_tags = train_test_split(sentences_X,sentence_tags_Y,test_size=0.2)

train_sentences,val_sentences,train_sentence_tags,val_sentence_tags = train_test_split(train_sentences,train_sentence_tags,test_size=0.2)

def generator(X,Y,batch_size):
    n_samples = len(X)
    n_batches = math.ceil(n_samples/batch_size)
    counter = 0 
    
    
    while 1:
        train_sentences_X_n = X[batch_size*counter:batch_size*(counter+1)]
#         test_sentences_X_n = test_sentences[batch_size*counter:batch_size*(counter+1)]
        train_sentence_tags_Y_n = Y[batch_size*counter:batch_size*(counter+1)]
#         test_sentence_tags_Y_n = test_sentence_tags[batch_size*counter:batch_size*(counter+1)]
        
        train_sentences_X_n = pad_sequences(train_sentences_X_n, maxlen=MAX_LENGTH, padding='post',value=np.zeros((1,300)))
#         test_sentences_X_n = pad_sequences(test_sentences_X_n, maxlen=MAX_LENGTH, padding='post',value=np.zeros((1,300)))
        train_sentence_tags_Y_n = pad_sequences(train_sentence_tags_Y_n, maxlen=MAX_LENGTH, padding='post',value=np.zeros((1,)))
#         test_sentence_tags_Y_n = pad_sequences(test_sentence_tags_Y_n, maxlen=MAX_LENGTH, padding='post',value=np.zeros((1,)))
        
        cat_train_tags_Y_n = to_categorical(train_sentence_tags_Y_n, len(tag2index)+1)
#         cat_test_tags_Y_n = to_categorical(test_sentence_tags_Y_n, len(tag2index)+1)
#         print(cat_train_tags_y[0].shape)
        counter +=1
        
        yield (train_sentences_X_n,cat_train_tags_Y_n)
        
        if counter <=n_batches:
            counter = 0
    

/usr/local/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
dataset = generator(train_sentences, to_categorical(train_sentence_tags, len(tag2index)+1),3000)

In [ ]:
a,b = next(dataset)

In [ ]:
%reset_selective -f "^b$"

In [ ]:
da

In [ ]:
from keras.preprocessing.sequence import pad_sequences

train_sentences = pad_sequences(train_sentences, maxlen=MAX_LENGTH, padding='post',value=np.zeros((1,300)))
test_sentences = pad_sequences(test_sentences, maxlen=MAX_LENGTH, padding='post',value=np.zeros((1,300)))
train_sentence_tags = pad_sequences(train_sentence_tags, maxlen=MAX_LENGTH, padding='post',value=np.zeros((1,)))
test_sentence_tags = pad_sequences(test_sentence_tags, maxlen=MAX_LENGTH, padding='post',value=np.zeros((1,)))[:2000]
 
# print(train_sentences[0])
# print(test_sentences[0])
# print(train_sentence_tags[0])
# print(test_sentence_tags[0])

In [ ]:

test_sentences.shape

In [ ]:
cat_train_tags_y = to_categorical(tr_Y, len(tag2index)+1)
print(cat_train_tags_y[0].shape)

In [ ]:
len(sentence_tags_Y)

## Train

In [27]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH,300)))
# model.add(Embedding(len(nepw2v.wv.vocab), 128))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(len(tag2index)+1)))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 148, 256)          570368    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 148, 111)          28527     
_________________________________________________________________
activation_1 (Activation)    (None, 148, 111)          0         
Total params: 598,895
Trainable params: 598,895
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.fit_generator(generator(train_sentences, to_categorical(train_sentence_tags, len(tag2index)+1),3000)
                    ,steps_per_epoch = math.ceil(sentences_X.shape[0]/3000), epochs=5,verbose=1)

Epoch 1/5


MemoryError: 

In [ ]:
to_categorical(train_sentence_tags, len(tag2index)+1).shape

##  Test model

In [ ]:
model.evaluate(test_sentences,to_categorical(test_sentence_tags,len(tag2index)+1))

In [ ]:
test = []
test_tags = []

x=[]
y=[]
for word in test1:
    x.append(word[1])
    y.append(word[0])
if len(x) > 0:
    test.append(x)
    test_tags.append(y)


In [ ]:
for i,x in enumerate(test_tags):
    test_tags[i] = [tag2index[t] for t in x]

In [ ]:
test_X = []
test_tags_Y = [] 
for i,sentence in enumerate(test):
    if len(sentence) > 0:
        sent = []
        sent_tag=[]
        for j,word in enumerate(sentence):
            try:
                sent.append(nepw2v.wv.get_vector(word))
                sent_tag.append(test_tags[i][j])
            except:
                continue

        test_X.append(np.array(sent))
        test_tags_Y.append(np.array(sent_tag))
test_X = np.array(test_X)
test_tags_Y = np.array(test_tags_Y)
    
# train_sentences,test_sentences,train_sentence_tags,test_sentence_tags  = train_test_split(sentences,sentence_tags,test_size=0.2)

In [ ]:
test_X = pad_sequences(test_X, maxlen=MAX_LENGTH, padding='post')
test_tags_Y = pad_sequences(test_tags_Y, maxlen=MAX_LENGTH, padding='post')

In [ ]:
test_X.shape

In [ ]:
predictions = model.predict_proba(test_X)

In [ ]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences
 

In [ ]:
g= logits_to_tokens(predictions, {i: t for t, i in tag2index.items()})[0]

for i,x in enumerate(test_tags[0]):
    print(g[i],test_tags[0][i])

In [ ]:
from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy
 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
 
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH,300 )))
# model.add(Embedding(len(tag2index), 128))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])
 
model.summary()
 

In [ ]:
model.fit(train_sentences, to_categorical(train_sentence_tags, len(tag2index)), batch_size=128, epochs=5, validation_split=0.2)

In [ ]:
predictions = model.predict(test_samples_X)
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))